In [1]:
from dicom_contour.contour import *
import matplotlib.pyplot as plt
import glob,os

In [15]:
path = './Breast_MRI_cont/Joly/'

In [16]:
def get_contour_file(path):
    """
    Get contour file from a given path by searching for ROIContourSequence 
    inside dicom data structure.
    More information on ROIContourSequence available here:
    http://dicom.nema.org/medical/dicom/2016c/output/chtml/part03/sect_C.8.8.6.html
    
    Inputs:
            path (str): path of the the directory that has DICOM files in it, e.g. folder of a single patient
    Return:
        contour_file (str): name of the file with the contour
    """
    # handle `/` missing
    if path[-1] != '/': path += '/'
    # get .dcm contour file
    fpaths = [path + f for f in os.listdir(path) if '.dcm' in f]
    n = 0
    for fpath in fpaths:
        f = dicom.read_file(fpath)
        if 'ROIContourSequence' in dir(f):
            contour_file = fpath.split('/')[-1]
            n += 1
    if n > 1: warnings.warn("There are multiple files, returning the last one!")
    return contour_file

In [23]:
#get the file name
contour_file = get_contour_file(path)

print(isinstance(contour_file, file))

#store its data
contour_data = dicom.read_file(path + '/' + contour_file)

get_roi_names(contour_data)

False


['Organ', 'GTV']

In [48]:
def cfile2pixels(file, path, ROIContourSeq=0):
    """
    Given a contour file and path of related images return pixel arrays for contours
    and their corresponding images.
    Inputs
        file: filename of contour
        path: path that has contour and image files
        ROIContourSeq: tells which sequence of contouring to use default 0 (RTV)
    Return
        contour_iamge_arrays: A list which have pairs of img_arr and contour_arr for a given contour file
    """
    # handle `/` missing
    if path[-1] != '/': path += '/'
    f = dicom.read_file(path + file)
    
    # index 0 means that we are getting RTV information
    RTV = f.ROIContourSequence[ROIContourSeq]
    # get contour datasets in a list
    contours = [contour for contour in RTV.ContourSequence]
    img_contour_arrays = [coord2pixels(cdata, path) for cdata in contours]
    return img_contour_arrays

In [60]:
# get all image-contour array pairs
contour_arrays = cfile2pixels(file=contour_file, path=path, ROIContourSeq=1)

IOError: [Errno 2] No such file or directory: './Breast_MRI_cont/Joly/1.2.840.113619.2.410.9231572.3304717.14769.1478086720.344.dcm'

In [61]:
print(path+'/'+contour_file)
f = dicom.read_file(path+'/'+contour_file)
print(f)


./Breast_MRI_cont/Joly//RS.1.2.246.352.71.4.139189879485.219264.20180517150349.dcm
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0012) Instance Creation Date              DA: '20180529'
(0008, 0013) Instance Creation Time              TM: '141321.265000'
(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.4.139189879485.219264.20180517150349
(0008, 0020) Study Date                          DA: '20161102'
(0008, 0030) Study Time                          TM: '113929'
(0008, 0050) Accession Number                    SH: 'RA2016340454'
(0008, 0060) Modality                            CS: 'RTSTRUCT'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: 'TREMBLAY, FRANCINE'
(0008, 1010) Station Name                        SH: 'WNETAP170'
(0008, 1030) Study Description                   LO: 'MRI BR

In [63]:

ROIContourSeq=1
RTV = f.ROIContourSequence[ROIContourSeq]
print(RTV)


(3006, 002a) ROI Display Color                   IS: ['255', '0', '0']
(3006, 0040)  Contour Sequence   7 item(s) ---- 
   (3006, 0016)  Contour Image Sequence   1 item(s) ---- 
      (0008, 1150) Referenced SOP Class UID            UI: MR Image Storage
      (0008, 1155) Referenced SOP Instance UID         UI: 1.2.840.113619.2.410.9231572.3304717.14769.1478086720.344
      ---------
   (3006, 0042) Contour Geometric Type              CS: 'CLOSED_PLANAR'
   (3006, 0046) Number of Contour Points            IS: '24'
   (3006, 0050) Contour Data                        DS: ['-75.66', '-110.03', '10.44', '-77.11', '-110.22', '10.44', '-77.62', '-110.54', '10.44', '-78.55', '-111.45', '10.44', '-78.89', '-111.98', '10.44', '-79.07', '-113.43', '10.44', '-79.27', '-114.87', '10.44', '-80', '-115.87', '10.44', '-80.2', '-116.32', '10.44', '-80', '-116.98', '10.44', '-79.76', '-117.77', '10.44', '-79.65', '-119.21', '10.44', '-78.55', '-120.23', '10.44', '-77.11', '-120.26', '10.44', '-76.07', 

In [69]:
RTV.ContourSequence
contours = [contour for contour in RTV.ContourSequence]